In [58]:
using TypedDelegation   
using StaticArrays
using Base.Test

In [2]:
using Tokenize
import Tokenize: Tokens
import Tokens: Token, startpos, endpos, untokenize, kind, exactkind



In [3]:
src = """
a=1
a+3
c=a+4; b=4

#A comment

function f(x)
    (1+2)/3
end

#= a 
multiline
comment
=#

println("abc ; efg")

\"""
a multiline
string
\"""

a+=2

map(1:5) do x
    a=2*x
    3*a
end

begin 
    c=1
end

if c
    d=3
end

"""
tokens = tokenize(src) |> collect;


In [4]:
immutable AnnotatedToken
    tok::Token
    notes::Dict{Symbol, Any}
end

AnnotatedToken(vv::Token) = AnnotatedToken(vv, Dict{Symbol, Any}())
@delegate_oneField(AnnotatedToken, tok, [exactkind, startpos, endpos, untokenize, kind])

kind (generic function with 2 methods)

In [5]:
?exactkind

search:



No documentation found.

`Tokenize.Tokens.exactkind` is a `Function`.

```
# 2 methods for generic function "exactkind":
exactkind(t::Tokenize.Tokens.Token) at /home/ubuntu/.julia/v0.5/Tokenize/src/token.jl:69
exactkind(a::AnnotatedToken, args...) at /home/ubuntu/.julia/v0.5/TypedDelegation/src/TypedDelegation.jl:69
```


In [6]:

function Base.show(io::IO, atok::AnnotatedToken)
  start_r, start_c = startpos(atok)
  end_r, end_c = endpos(atok)
  str = kind(atok) == Tokens.ENDMARKER ? "" : untokenize(atok)
    print(io, start_r, ",", start_c, "-",
            end_r,   ",", end_c,   ":",
    "   ", exactkind(atok), " (",lowercase(string(kind(atok))),")","\t")
    show(io, str)
end

In [7]:
display.(AnnotatedToken.(tokens));

1,1-1,1:   IDENTIFIER (identifier)	"a"

1,2-1,2:   EQ (op)	"="

1,3-1,3:   INTEGER (integer)	"1"

1,4-2,0:   WHITESPACE (whitespace)	"\n"

2,1-2,1:   IDENTIFIER (identifier)	"a"

...

35,7-35,7:   INTEGER (integer)	"3"

35,8-36,0:   WHITESPACE (whitespace)	"\n"

36,1-36,3:   END (keyword)	"end"

36,4-38,0:   WHITESPACE (whitespace)	"\n\n"

38,1-38,0:   ENDMARKER (endmarker)	""

- **New line** always ends expressions
 - Unless Immediately within a block opened by `(`
 - Or where the last nonwhitespace token was binary operator (including a `,`?)

In [62]:
isnewline(tok) = kind(tok)==Tokens.WHITESPACE && '\n' ∈ untokenize(tok)


isnewline (generic function with 1 method)

In [63]:
@test isnewline(tokens[end-1])

Test Passed
  Expression: isnewline(tokens[end - 1])

is_ender (generic function with 1 method)

In [70]:
const identifier_phrase_end_kinds = SVector[Tokens.IDENTIFIER, Tokens.RPAREN, Tokens.RSQUARE]

LoadError: cannot declare identifier_phrase_end_kinds constant; it already has a value

In [8]:
function annotate(toks)
    subclause_markers = Dict(
            Tokens.LBRACE => Tokens.RBRACE,
            Tokens.LPAREN => Tokens.RPAREN,
            Tokens.LSQUARE => Tokens.RSQUARE,
            Tokens.FOR => Tokens.END,
            Tokens.WHILE => Tokens.END,
            Tokens.IF => Tokens.END,
            Tokens.TRY => Tokens.END,
            Tokens.BEGIN => Tokens.END,
            Tokens.QUOTE => Tokens.END
            
    )
    open_subclauses = AnnotatedToken[]
    next_closer_exactkind() = subclause_markers[exactkind(open_subclauses[end])]
    
    in_paren() = exactkind(open_subclauses[end]) == Tokens.LPAREN
    in_square() = exactkind(open_subclauses[end]) == Tokens.LSQUARE
    

    
    
    
    history = SVector(Tokens.ENDMARKER, Tokens.ENDMARKER)
    function add_history!(atok)
        if kind(atok) != Tokens.WHITESPACE || isnewline(tok)
            #TODO: Consider replacing this with somehting faster
            @assert(!isnewline(tok) || !isnewline(history[end]), "New lines should not follow each other -- those should have been merged by tokenizer")
            history = @SVector([shift(history)..., kind(atok)])
        end
    end
    
    am_continuing_operation() = history[end]==Tokens.OP && history[end-1] ∈ identifier_phrase_end_kinds

    
    function handle_ifclose!(atok)
        if  next_closer_exactkind() == exactkind(atok)
            opener = pop!(open_subclauses)
            opener.notes[:closer] = atok
            atok.notes[:opener] = opener
        end
    end
    
    function handle_ifopen!(atok)
        if haskey(subclause_markers, exactkind(atok))
            push!(open_subclauses, atok)
        end
    end
    
    
    "Handless expression enders, (this excludes closers)."
    function handle_expression_enders!(atok)
        is_ender = ((!in_paren() &&  !am_continuing_operation() && isnewline(atok)) 
                        || exactkind(atok) == Tokens.SEMICOLON)    
        if is_ender
            atok.notes[:ender] = true
        end
    end
    
    
    outstream = AnnotatedToken[]
    for tok in toks
        atok = Annotated(tok)
        push!(outstream, atok)
                
        handle_ifopen(atok)
        handle_ifclose(atok)
        handle_expression_enders!(atok)
        
        add_history!(atok)         
    end
    outstream
end

annotate (generic function with 1 method)

In [72]:
(tokenize("""
(a +
b)
{a +
b}
[a +
b]


""") |> collect )

25-element Array{Tokenize.Tokens.Token,1}:
 1,1-1,1:   LPAREN	"("         
 1,2-1,2:   IDENTIFIER	"a"     
 1,3-1,3:   WHITESPACE	" "     
 1,4-1,4:   OP	"+"             
 1,5-2,0:   WHITESPACE	"\n"    
 2,1-2,1:   IDENTIFIER	"b"     
 2,2-2,2:   RPAREN	")"         
 2,3-3,0:   WHITESPACE	"\n"    
 3,1-3,1:   LBRACE	"{"         
 3,2-3,2:   IDENTIFIER	"a"     
 3,3-3,3:   WHITESPACE	" "     
 3,4-3,4:   OP	"+"             
 3,5-4,0:   WHITESPACE	"\n"    
 4,1-4,1:   IDENTIFIER	"b"     
 4,2-4,2:   RBRACE	"}"         
 4,3-5,0:   WHITESPACE	"\n"    
 5,1-5,1:   LSQUARE	"["        
 5,2-5,2:   IDENTIFIER	"a"     
 5,3-5,3:   WHITESPACE	" "     
 5,4-5,4:   OP	"+"             
 5,5-6,0:   WHITESPACE	"\n"    
 6,1-6,1:   IDENTIFIER	"b"     
 6,2-6,2:   RSQUARE	"]"        
 6,3-9,0:   WHITESPACE	"\n\n\n"
 9,1-9,0:   ENDMARKER	""       

In [67]:
(a,b) = 
3,4

(3,4)

In [35]:
unshift!(history, Tokens.KEYWORD)

LoadError: MethodError: no method matching unshift!(::StaticArrays.SVector{2,Tokenize.Tokens.Kind}, ::Tokenize.Tokens.Kind)[0m
Closest candidates are:
  unshift!(::Any, ::Any, [1m[31m::Any[0m) at abstractarray.jl:1717
  unshift!(::Any, ::Any, [1m[31m::Any[0m, [1m[31m::Any...[0m) at abstractarray.jl:1718
  unshift!{T}([1m[31m::Array{T,1}[0m, ::Any) at array.jl:537
  ...[0m

2-element StaticArrays.SVector{2,Tokenize.Tokens.Kind}:
 IDENTICAL_WITH_DOT_ABOVE
 KEYWORD                 

In [51]:
methodswith(StaticArray)

99-element Array{Method,1}:
 *{TA<:Union{Complex{Float32},Complex{Float64},Float32,Float64},Tb}(A::StaticArrays.StaticArray{TA,2}, b::StaticArrays.StaticArray{Tb,1}) at /home/ubuntu/.julia/v0.5/StaticArrays/src/matrix_multiply.jl:134                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
 *{TA,Tb}(A::StaticArrays.StaticArray{TA,2}, b::StaticArrays.StaticArray{Tb,1}) at /home/ubuntu/.julia/v0.5/StaticArrays/src/matrix_multiply.jl:95                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
 *{TA<:Union{Complex{Float32},Complex{Float64},Float32,Float64},Tb}(A::StaticArrays.StaticArray{TA,2}, b::Union{Base.ReshapedArray{Tb,1,A<:DenseArray,MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64},N<:Any}}},DenseArray{Tb,1},SubArray{Tb,1,A<:Union{Base.ReshapedArray{T<:Any,N<:Any,A<:DenseArray,MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64},N<:Any}}},DenseArray},I<:Tuple{Vararg{Union{Base.AbstractCartesianIndex,Colon,Int64,Range{Int64}},N<:Any}},L<:Any}}) at /home/ubuntu/.julia/v0.5/StaticArrays/src/matrix_multiply.jl:202                                                                                                                                                                                                                                                                                                                                                                                                                 
 *(a::StaticArrays.StaticArray, b::Number) at /home/ubuntu/.julia/v0.5/StaticArrays/src/linalg.jl:26                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            


In [11]:
begin
    a=2
    b=3
end

3

In [12]:
0,
2

(0,2)

In [13]:
:(a=2;
c=4
b=3) |> dump

LoadError: syntax: missing comma or ) in argument list

In [14]:
[a=2
b=3]

2-element Array{Int64,1}:
 2
 3

In [15]:
a=2
(a
+1
)

3

In [16]:
a=2
(
if a>1
    a
    +1
end
)


1

In [17]:
a=2
(
begin
    a
    +1
end
)


1

In [18]:
a=2
begin
    a
    +1
end



1

In [19]:
a=2
(
[
    a
    +1
]
)


2-element Array{Int64,1}:
 2
 1

In [20]:
a=2
[
    (
    a
    +1
    )
]



1-element Array{Int64,1}:
 3

In [21]:
x=[10,20,30]
a=2
x[a+1] |> display



30

In [22]:
x[(a+1)  
    
] |> display

LoadError: MethodError: no method matching typed_vcat(::Array{Int64,1}, ::Int64)[0m
Closest candidates are:
  typed_vcat{T}([1m[31m::Type{T}[0m, ::Number...) at abstractarray.jl:971
  typed_vcat([1m[31m::Type{T}[0m, ::Any...) at abstractarray.jl:1182[0m

In [23]:
x[
    (
    a
    +1
    )
] |> display

30

In [24]:
x=[10,20,30]
x[(2
    
    )
]

20

In [25]:
x=rand(4,4)
x[ 2;2    
]

LoadError: MethodError: no method matching typed_vcat(::Array{Float64,2}, ::Int64, ::Int64)[0m
Closest candidates are:
  typed_vcat{T}([1m[31m::Type{T}[0m, ::Number...) at abstractarray.jl:971
  typed_vcat([1m[31m::Type{T}[0m, ::Any...) at abstractarray.jl:1182[0m